In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.graph_objects as go 
import sys
# Adjust the path below to the path of your `module_directory`
sys.path.append('../thesis-2p-behavior-align/')
# Now you can import your module or function
import imaging_behavior_functions
import odor_behavioral_analysis
import scipy.cluster.hierarchy as sch
from rastermap import Rastermap, utils
from scipy.stats import zscore
from scipy.stats import sem
from scipy.spatial import KDTree

In [2]:
base_path = "//research.files.med.harvard.edu/neurobio/wilsonlab/Jingxuan/stacks/"
example_path_data = base_path+"20231205-4_hDeltaB_tomogcamp7f_5dur75isi_fly2/data/"
example_path_results = base_path+"20231205-4_hDeltaB_tomogcamp7f_5dur75isi_fly2/results/"
trial_num = 1
odor_threshold = 5
time_interval_threshold = 16  # Assuming time is in seconds or an equivalent unit
k = 8
window_size = 30

is_mat73, roi_df, dff_raw, kinematics_raw, preprocessed_vars_ds, preprocessed_vars_odor = imaging_behavior_functions.load_intermediate_mat(example_path_data,trial_num)
behav_df = imaging_behavior_functions.make_df_behavior(dff_raw, preprocessed_vars_ds, preprocessed_vars_odor,trial_num,ball_d = 9)
xPos, yPos = imaging_behavior_functions.reconstruct_path(behav_df, ball_d = 9)
roi_names, hdeltab_index, epg_index, hdeltab_sequence, epg_sequence = imaging_behavior_functions.get_roi_seq(roi_df)
dff_all_rois, dff_time = imaging_behavior_functions.load_dff_raw(is_mat73, dff_raw)
neural_df = imaging_behavior_functions.make_df_neural(dff_all_rois, dff_time, roi_names, hdeltab_index, epg_index, hdeltab_sequence, epg_sequence)
behav_df, padded_result_df, smoothed_df = odor_behavioral_analysis.analysis_dfs(behav_df, time_interval_threshold, k, window_size)
combined_df = imaging_behavior_functions.combine_df(behav_df, neural_df)
smoothed_df["translationalV_smoothed"] = np.sqrt(smoothed_df["fwV_smoothed"]**2+smoothed_df["sideV_smoothed"]**2)

In [3]:
selected_df = neural_df.drop(columns=['time'])
selected_df

,hDeltaB_0,hDeltaB_1,hDeltaB_2,hDeltaB_3,hDeltaB_4,hDeltaB_5,hDeltaB_6,hDeltaB_7,hDeltaB_8,hDeltaB_9,hDeltaB_10,hDeltaB_11
0,0.067182,0.160227,0.081253,0.009190,0.019644,0.003423,0.015684,-0.047264,-0.004261,0.065573,0.128344,0.267302
1,0.113393,0.156151,0.075551,0.046179,0.013253,-0.015796,-0.006278,-0.035016,0.004829,0.045642,0.114370,0.338818
2,0.150793,0.161208,0.072277,0.075688,0.025626,-0.020140,-0.010499,-0.026168,0.021801,0.046403,0.104895,0.381662
3,0.156863,0.175518,0.077826,0.083740,0.058145,-0.000332,0.000631,-0.011459,0.049131,0.061871,0.094829,0.365157
4,0.128071,0.168776,0.082242,0.074708,0.069190,0.014238,0.006162,-0.004507,0.050148,0.055686,0.083402,0.302955
...,...,...,...,...,...,...,...,...,...,...,...,...
3490,0.120887,0.168595,0.117021,0.146620,0.175909,0.180057,0.159247,0.091957,0.117306,0.096745,0.110315,0.210445
3491,0.156958,0.162354,0.119769,0.103034,0.154810,0.156696,0.125775,0.089120,0.100946,0.094543,0.081195,0.234688
3492,0.214150,0.143146,0.118893,0.083091,0.155062,0.144924,0.115857,0.103902,0.094422,0.095494,0.094089,0.263892
3493,0.199519,0.116515,0.101906,0.076322,0.147625,0.104345,0.099341,0.098168,0.097290,0.083225,0.091479,0.251051


In [4]:
roi_kw = 'hDeltaB'
roi_mtx = imaging_behavior_functions.extract_heatmap(neural_df, roi_kw, 1, example_path_results, trial_num)
param_df = imaging_behavior_functions.fit_sinusoid(neural_df,roi_mtx)
param_df

,time,phase,baseline,amplitude,phase_error,baseline_error,amplitude_error
0,0.000000,0.000000,-0.976340,0.824229,0.213832,0.120547,0.164510
1,0.171689,0.000000,-0.957066,0.980153,0.142866,0.095862,0.130961
2,0.343377,0.000000,-0.868282,1.031191,0.149921,0.105869,0.144682
3,0.515066,0.000000,-0.731641,0.887192,0.165593,0.100656,0.137633
4,0.686754,0.571199,-0.758464,0.719398,0.163581,0.080973,0.111259
...,...,...,...,...,...,...,...
3490,599.193064,2.855993,0.023821,0.390032,0.282704,0.075953,0.104487
3491,599.364752,2.284795,-0.121618,0.217824,0.422032,0.064369,0.090127
3492,599.536441,2.284795,-0.109355,0.075390,1.320773,0.070645,0.100242
3493,599.708129,0.571199,-0.251663,0.022362,3.906305,0.060520,0.083793


In [5]:
base_dir = '//research.files.med.harvard.edu/neurobio/wilsonlab/Jingxuan/stacks/20231205-4_hDeltaB_tomogcamp7f_5dur75isi_fly2/E_05_Iter_10368/plane_4/suite2p/plane0'
F = np.load(base_dir+'/F.npy', allow_pickle=True)
Fneu = np.load(base_dir+'/Fneu.npy', allow_pickle=True)
spks = np.load(base_dir+'/spks.npy', allow_pickle=True)
stat = np.load(base_dir+'/stat.npy', allow_pickle=True)
ops =  np.load(base_dir+'/ops.npy', allow_pickle=True)
ops = ops.item()
iscell = np.load(base_dir+'/iscell.npy', allow_pickle=True)

In [6]:
F_cell = F[(iscell[:,0]==1),:]
spks_cell = spks[(iscell[:,0]==1),:]
Fneu_cell = Fneu[(iscell[:,0]==1),:]
stat_cell = stat[(iscell[:,0]==1)]